In [20]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI  # Updated import
from dotenv import load_dotenv
from typing import TypedDict, List
from pydantic import BaseModel, Field
import operator

In [13]:
load_dotenv()

True

In [14]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [15]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description='detaild feedback for the essay')
    score: int = Field(description='Score out of 10',ge=0,le=10)
    

In [16]:
structured_model = model.with_structured_output(EvaluationSchema)

In [17]:
essay = """
        Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century, reshaping industries, economies, and societies through automation, data analysis, and intelligent decision-making. As the world races to harness AI's potential, India stands out as a key player, leveraging its vast talent pool, government initiatives, and strategic investments to position itself as a global AI powerhouse. With a population exceeding 1.4 billion and a burgeoning digital economy, India's role in AI is not merely participatory but increasingly influential, driving innovation that addresses both domestic challenges and global needs. This essay explores India's contributions to AI, from policy frameworks and industry advancements to talent development and future prospects, highlighting how the nation is shaping the AI landscape.
        India's journey in AI began gaining momentum in the late 2010s, with foundational efforts to integrate the technology into national development. In 2018, the National Institution for Transforming India (NITI Aayog), the government's premier think tank, released the National Strategy for Artificial Intelligence, titled "#AIforAll." This document identified key sectors such as healthcare, agriculture, education, smart cities, and transportation where AI could drive inclusive growth. Recognizing India's unique position as the world's fastest-growing major economy and second-most populous nation, the strategy emphasized AI's potential to solve pressing issues like poverty alleviation and resource optimization. By 2023, this vision evolved into the IndiaAI Mission, a comprehensive program backed by approximately USD 1.2 billion in funding to build an ecosystem fostering AI innovation. The mission focuses on democratizing access to computing infrastructure, datasets, and talent, ensuring AI benefits reach underserved populations.
        Government initiatives have been pivotal in propelling India's AI agenda. The Union Budget for 2025-2026 allocated INR 20,000 crores (about USD 2.4 billion) to AI research and development, signaling a commitment to long-term growth. Programs like Digital India and the Global Partnership on Artificial Intelligence (GPAI), where India served as chair, underscore ethical AI development and international collaboration. Prime Minister Narendra Modi's administration has emphasized AI's role in governance, with initiatives like AI-powered public services and the AI Action Summit in Paris, where India co-chaired discussions on global standards.
        """

In [18]:
prompt = f'''Evaluate the language quality of the following essay and provide feedback and assign a score out of 10\n {essay}'''
structured_model.invoke(prompt).feedback # type: ignore

"The essay demonstrates excellent language quality, characterized by clear, concise, and coherent writing. The introduction effectively sets the stage, and the subsequent paragraphs are well-structured, providing strong evidence and examples to support India's contributions to AI, such as the NITI Aayog's '#AIforAll' strategy, the IndiaAI Mission, and significant budget allocations. The vocabulary is appropriate and sophisticated, enhancing the professional tone. There are no noticeable grammatical errors or awkward phrasing, contributing to a smooth and engaging reading experience. The essay effectively communicates complex information about India's AI landscape, showcasing a solid grasp of the subject matter and an ability to articulate it persuasively."

In [ ]:
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[List[int], operator.add]
    avg.score: Float

NameError: name 'Annotated' is not defined

In [ ]:
def evalaute_language(state: UPSCState):

    prompt = f'''Evaluate the language quality of the following essay and provide feedback and assign a score out of 10\n {state["essay"]}'''
    output = structured_model.invoke(prompt)

    return{'language_feedback': output.feedback, 'individual_scores': {output.score}}

In [ ]:
def evalaute_analysis(state: UPSCState):

    prompt = f'''Evaluate the depth of analysis of the following essay and provide feedback and assign a score out of 10\n {state["essay"]}'''
    output = structured_model.invoke(prompt)

    return{'analysis_feedback': output.feedback, 'individual_scores': {output.score}}

In [ ]:
def evalaute_thought(state: UPSCState):

    prompt = f'''Evaluate the clarity of thought of analysis of the following essay and provide feedback and assign a score out of 10\n {state["essay"]}'''
    output = structured_model.invoke(prompt)

    return{'clarity_feedback': output.feedback, 'individual_scores': {output.score}}

In [ ]:
def final_evaluation(state: UPSCState):
    #Summary Feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content
    
    #Avg calcluate

    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback':overall_feedback, 'avg_score':avg_score}


In [ ]:
graph  = StateGraph(UPSCState)

graph.add_node('evaluate_langauge',evaluate_langauge)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)


#Graph Edges

graph.add_edge(START, 'evaluate_langauge')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_langauge', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')


graph.add_edge('final_evaluation', END)

workflow = graph.compile

In [ ]:
intial_state = {
    'essay' : essay
}

workflow.invoke(intial_state)